In [20]:
import pickle
import os
import json
import pandas as pd
from tqdm import tqdm
from collections import Counter, defaultdict

In [3]:
output_path = '../data/outputs/qwen-3-235b-a22b-instruct-2507/'

In [ ]:
# 1. map json with excel
# 2. 

In [11]:
import json

with open('../data/outputs/qwen-3-235b-a22b-instruct-2507/CN101033114A.json', 'r') as f:
    data = json.load(f)


In [14]:
import ast

In [36]:
from json_repair import repair_json

# good_json_string = repair_json(bad_json_string)

In [62]:
js = 'CN101033114A.json'
with open(f'../data/outputs/qwen-3-235b-a22b-instruct-2507/{js}', 'r') as f:
    data = json.load(f)['answer']

In [63]:
column_dict = dict()

files = os.listdir('../data/excel/')
files.remove('.DS_Store')

mapper = dict()
ex = js2excel[js]

df = pd.read_excel(f'../data/excel/{ex}')
columns = df.columns.tolist()
idx = df[df[columns[0]] == 'Glass No'].index[-1]

idx0 = df[df[columns[0]] == 'Glass No'].index[0]
idx1 = df[df[columns[0]] == 'Glass No'].index[-1]

for k,v in zip(df.iloc[idx0], df.iloc[idx1]):
    k = k.replace("\n", ' ')
    if k in mapper:
        mapper[k].append(v)
    else:
        mapper[k] = [v]
dff = df.iloc[idx:,:]
dff.columns = df.iloc[idx]
dff = dff.iloc[1:,:]
column_dict[file] = dff.columns.tolist()

In [71]:
gold = []
for i in range(len(dff)):
    gold.append(dff.iloc[i].to_dict())

In [74]:
gold[2]

{'Glass No': 415124,
 'SiO2': 54.55,
 'Al2O3': 14.6,
 'B2O3': 26.3,
 'TiO2': 1,
 'CaO': 1,
 'MgO': 2,
 'Na2O': 0.1,
 'K2O': 0.1,
 'F': 0.2,
 'V2O5': 0.1,
 'CeO2': 0.05,
 'α*1E7, K-1 temp.range, °C in 100-400': 27.8,
 'T, °C at log(η, P) at 3': 1335,
 "ε' f, MHz at 1": 4.1,
 'Tanδ*1E4 f, MHz at 1': 6.6,
 "ε'": 4.1,
 'Tanδ*1E4': 6.6}

In [75]:
ast.literal_eval(data)[0]

[{'SiO2': 54.55,
  'Al2O3': 14.6,
  'B2O3': 26.3,
  'TiO2': 1.0,
  'CaO': 1.0,
  'MgO': 2.0,
  'Na2O': 0.1,
  'K2O': 0.1,
  'F': 0.2,
  'V2O5': 0.1,
  'CeO2': 0.05},
 {'dielectric_constant': [{'value': 4.1,
    'unit': '',
    'experimental_conditions': 'room temperature, 1MHz'}],
  'dielectric_tangent': [{'value': 0.00066,
    'unit': '',
    'experimental_conditions': 'room temperature, 1MHz'}],
  'thermal_expansion': [{'value': 2.78e-06,
    'unit': '/℃',
    'experimental_conditions': '100-400℃'}],
  'viscosity': [{'value': 1335,
    'unit': '℃',
    'experimental_conditions': '10^3 poise'}]}]

In [52]:
outputs = []
for ff in tqdm(os.listdir(output_path)):
    with open(f'../data/outputs/qwen-3-235b-a22b-instruct-2507/{ff}', 'r') as f:
        data = json.load(f)['answer']
        outputs.append(data)
        

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 55/55 [00:00<00:00, 3833.28it/s]


In [58]:
column_dict = dict()

files = os.listdir('../data/excel/')
files.remove('.DS_Store')

mapper = dict()

for file in :
    df = pd.read_excel(f'../data/excel/{file}')
    columns = df.columns.tolist()
    idx = df[df[columns[0]] == 'Glass No'].index[-1]
    
    idx0 = df[df[columns[0]] == 'Glass No'].index[0]
    idx1 = df[df[columns[0]] == 'Glass No'].index[-1]

    for k,v in zip(df.iloc[idx0], df.iloc[idx1]):
        k = k.replace("\n", ' ')
        if k in mapper:
            mapper[k].append(v)
        else:
            mapper[k] = [v]
    dff = df.iloc[idx:,:]
    dff.columns = df.iloc[idx]
    dff = dff.iloc[1:,:]
    column_dict[file] = dff.columns.tolist()

In [57]:
ast.literal_eval(outputs[5])#:7]

[[{'SiO2': 70.619,
   'Al2O3': 6.0,
   'Na2O': 17.0,
   'CaO': 2.0,
   'MgO': 0.01,
   'F': 4.0,
   'P2O5': 0.1,
   'Fe2O3': 0.1,
   'SO3': 0.1,
   'Co2O3': 0.005,
   'CuO': 0.005,
   'Cr2O3': 0.001,
   'MnO': 0.005,
   'NiO': 0.05,
   'Se': 0.005},
  {'softening_temperature': [{'value': 620,
     'unit': '°C',
     'experimental_conditions': ''}]}],
 [{'SiO2': 62.64,
   'Al2O3': 3.0,
   'Na2O': 15.0,
   'CaO': 12.0,
   'MgO': 5.0,
   'F': 1.0,
   'P2O5': 1.0,
   'Fe2O3': 0.01,
   'SO3': 0.3,
   'Co2O3': 0.05},
  {'softening_temperature': [{'value': 630,
     'unit': '°C',
     'experimental_conditions': ''}]}],
 [{'SiO2': 60.0,
   'Al2O3': 7.0,
   'Na2O': 10.0,
   'CaO': 12.0,
   'MgO': 3.55,
   'F': 4.0,
   'P2O5': 2.0,
   'Fe2O3': 0.5,
   'SO3': 0.7,
   'NiO': 0.25},
  {'softening_temperature': [{'value': 650,
     'unit': '°C',
     'experimental_conditions': ''}]}],
 [{'SiO2': 65.65,
   'Al2O3': 5.0,
   'Na2O': 12.0,
   'CaO': 6.0,
   'MgO': 4.0,
   'F': 3.0,
   'P2O5': 3.0,
   'F

In [24]:
jsons = os.listdir(output_path)
excels = os.listdir('../data/excel/')
excels.remove('.DS_Store')

In [31]:
js2excel = dict()
excel2js = dict()

for js, ex in zip(sorted(jsons), sorted(excels)):
    js2excel[js] = ex
    excel2js[ex] = js

In [33]:
js2excel = {
 'CN101033114A.json': 'CN101033114 _table-33959.xlsx',
 'CN101058477A.json': 'CN101058477_table-35472.xlsx',
 'CN101072676A.json': 'CN101072676A_table-30659.xlsx',
 'CN101092308B.json': 'CN101092308B_table-35489.xlsx',
 'CN1308591A.json': 'CN1308591_table-24104.xlsx',
 'CN1326903A.json': 'CN1326903_table-28556.xlsx',
 'CN1489965A.json': 'CN1489965_table-37874.xlsx',
 'CN1495138A.json': 'CN1495138_table-30623.xlsx',
 'CN1522980A.json': 'CN1522980_table-28312.xlsx',
 'CN1771211A.json': 'CN1771211_table-28366.xlsx',
 'FR_1027316_A.json': 'FR1027316_table-16101.xlsx',
 'FR_1172977_A.json': 'FR1172977_table-29832.xlsx',
 'FR_1242292_A.json': 'FR1242292_table-37871.xlsx',
 'FR_1247560_A.json': 'FR1247560_table-29840.xlsx',
 'FR_1388413_A.json': 'FR1388413_table-17489.xlsx',
 'FR_1484178_A.json': 'FR1484178_table-30674.xlsx',
 'FR_2511360_A1.json': 'FR2511360_table-8590.xlsx',
 'FR_2564087_A1.json': 'FR2564087_table-8498.xlsx',
 'FR_2650266_A1.json': 'FR2650266_table-4410.xlsx',
 'FR_2775476_A1.json': 'FR2775476_table-17437.xlsx',
 'JP2013209231A.json': 'JP2013209231_table-41689.xlsx',
 'JP2013520387A.json': 'JP2013520387_table-39169.xlsx',
 'JP2013520388A.json': 'JP2013520388_table-43286.xlsx',
 'JP2014019632A.json': 'JP2014019632_table-41679.xlsx',
 'JP2014504250A.json': 'JP2014504250_table-39897.xlsx',
 'JP2015013773A.json': 'JP2015013773_table-42776.xlsx',
 'JP2015509903A.json': 'JP2015509903_table-43872.xlsx',
 'JP5454585B2.json': 'JP5454585_table-40257.xlsx',
 'JP5613164B2.json': 'JP5613164_table-40102.xlsx',
 'JP5743125B2.json': 'JP5743125_table-43288.xlsx',
 'JP5763852B2.json': 'JP5763852_table-44883.xlsx',
 'KR20160040680A.json': 'KR20160040680_table-42977.xlsx',
 'KR20170007352A.json': 'KR20170007352_table-43529.xlsx',
 'KR20170021584A.json': 'KR20170021584_table-43757.xlsx',
    'RU2016861C1.json': 'RU2016861_table-4306.xlsx',
    'RU2017695C1.json': 'RU2017695_table-4488.xlsx',
    'RU2045486C1.json': 'RU2045486_table-15262.xlsx',
    'RU2069198C1.json': 'RU2069198_table-15265.xlsx',
    'RU2102345C1.json': 'RU2102345_table-2783.xlsx',
    'RU_2015120_C1.json':'RU2015120_table-15217.xlsx',
    'RU_2016858_C1.json': 'RU2016858_table-4323.xlsx',
    'RU_2021219_C1.json': 'RU2021219_table-4489.xlsx',
    'RU_2044710_C1.json': 'RU2044710_table-8002.xlsx',
    'RU_2383502_C1.json': 'RU2383502_table-37098.xlsx',
 'US1304623.json': 'US1304623_table-24764.xlsx',
 'US1623301.json': 'US1623301_table-24156.xlsx',
 'US20010014424A1.json': 'US20010014424A1_table-33582.xlsx',
 'US20030228968A1.json': 'US20030228968A1_table-28530.xlsx',
 'US20040214047A1.json': 'US20040214047A1_table-28846.xlsx',
 'US20060166804A1.json': 'US20060166804A1_table-33051.xlsx',
 'US20090239122A1.json': 'US2009239122A1_table-30659.xlsx',
 'US2486812.json':'US2486812_table-16059.xlsx',
 'US2920971.json':'US2920971_table-37870.xlsx',     
  
}

excel2js = dict()


    
    # 'US3640738_table-13146.xlsx', 'US20090286667A1.json': ,